In [1]:
#导入各种需要的库
import geemap
import ee
Map = geemap.Map()
from IPython.display import Image
import PySimpleGUI as sg
from tqdm.notebook import tqdm
from time import sleep
import csv
import logging
def deleteContent(fName):
    with open(fName, "w"):
        pass
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [*] %(processName)s %(message)s"
)
deleteContent('竞赛.csv')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [2]:

xiajiang_shp = r"G:\competition\xiajiang\峡江县.shp" #添加矢量数据
xiajiang = geemap.shp_to_ee(xiajiang_shp)    
Map.centerObject(xiajiang, zoom=10)  #将矢量数据添加到底图中
roi = xiajiang.geometry()  #获取矢量的范围
 #定义双季水稻种植的时间范围
Double_1_1 = ['04-01','05-10']    #早稻移栽期
Double_1_2 = ['05-11','06-10']    #早稻拔节期
Double_1_3 = ['06-11','07-10']    #早稻抽穗期
Double_2_1 = ['07-11','08-20']    #晚稻移栽期
Double_2_2 = ['08-21','09-20']    #晚稻拔节期
Double_2_3 = ['09-21','10-20']    #晚稻抽穗期
Names = ['yizai1','bajie1','chousui1','yizai2','bajie2','chousui2']   #输出数据的名称
D_list = [Double_1_1,Double_1_2,Double_1_3,Double_2_1,Double_2_2,Double_2_3]  


#定义的一些可视化参数
palette2 = ['FFFFFF,CE7E45,DF923D,F1B555,FCD163,99B718,74A901']
ndvi_visparam = {'min':-1,'max':1,'palette':['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']}
L5_visualization = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'],'min': 0.0,'max': 0.3}
L7_visualization = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'],'min': 0.0,'max': 0.3}
L8_visualization = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0.0,'max': 0.3}
elev_args = {'crs': 'EPSG:3857','dimensions': '300','region': roi,'min': -2000,'max': 10000,'palette': 'green, blanchedalmond,orange,black ','framesPerSecond': 12}
classVis = {'min': 0,'max': 10,'palette': ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4','f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0']}
sg.popup_notify(title = 'Task done!',display_duration_in_ms = 100,fade_in_duration = 100)

'__TIMEOUT__'

In [3]:

LUCC = ee.ImageCollection("ESA/WorldCover/v100")   #添加土地利用数据  https://viewer.esa-worldcover.org/worldcover/?language=en&bbox=-300.05859375,-84.03431939673978,300.05859375,84.03431939673979&overlay=true&bgLayer=MapBox_Satellite&date=2022-08-08&layer=WORLDCOVER_2020_MAP
#Zanaga, D., Van De Kerchove, R., De Keersmaecker, W., Souverijns, N., Brockmann, C., Quast, R., Wevers, J., Grosu, A., Paccini, A., Vergnaud, S.，Cartus，O.，Santoro，M.，Fritz，S.，Georgieva，I.，Lesiv，M.，Carter，S.，Herold，M.，Li，Linlin，Tsendbazar，NE，Ramoino，F。 , Arino, O., 2021. ESA WorldCover 10 m 2020 v100。https://doi.org/10.5281/zenodo.5571936 
LUCC = LUCC.filterBounds(roi).first()   #获取影像集中的第一张图像
classValues = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]   #映射图像中的值：将10，20，30，40--70，80，90映射为0-10
remapValues = ee.List.sequence(0, 10)   
label = 'lc'
LUCC = LUCC.remap(classValues, remapValues).rename(label).toByte() #将输入值转换为一个无符号的8位整数。
'''添加影像集'''
Landsat8_col = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")  #2013-2021  L8影像信息为：https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2?hl=en
Landsat7_col = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")  #2000-2021  L7影像信息为：https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C02_T1_L2?hl=en
Landsat5_col = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")  #1984-2011  L5影像信息为：https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C02_T1_L2?hl=en
Sentinel_col  = ee.ImageCollection("COPERNICUS/S2_SR")  #2017-2021  哨兵影像信息为：https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_Global_SRTM_landforms?hl=en#description
SRTM30 = ee.Image("USGS/SRTMGL1_003").select('elevation')   #影像信息为： https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_Global_SRTM_landforms?hl=en
slope = ee.Terrain.slope(SRTM30).clip(roi).rename('slope')  #由高程数据计算坡度

'''Add layer'''
Map.addLayer(SRTM30,elev_args,'SRTM30')  #将各种数据添加至图层中
Map.addLayer(LUCC, classVis, "LUCC")
Map.addLayer(roi, {'color': 'white'}, 'ROI', True, 0.5)
sg.popup_notify(title = 'Task done!',display_duration_in_ms = 100,fade_in_duration = 100)

'__TIMEOUT__'

In [ ]:
'''定义裁剪函数'''
def clipp(image):
     return image.clip(roi)

'''定义缩放比例的函数'''
def L8_applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)    
def Sn_applyScaleFactors(image):
    opticalBands = image.select('B.*').multiply(0.0001)
    thermalBands = image.select(['AOT','WVP']).multiply(0.001)
    return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)    
def L7_applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
              .addBands(thermalBand, None, True)
def L5_applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)    

'''定义L8,L7,L5,哨兵卫星的去云函数'''
def L8_maskL8sr(image):
      #<<表示二进制左移几位
      # 二进制第三位表示有云阴影，第五位表示有云。
    cloudShadowBitMask = 1 << 4
    cloudsBitMask = 1 << 3
    #选择质量评估波段
    qa = image.select('QA_PIXEL')
    #按位与运算，将检测出云以及云阴影像元值为0
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)\
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    #掩膜云以及云阴影，并选择波段1-9将单位变为0-1之间，并复制影像时间属性。
    return image.updateMask(mask) \
      .select("SR_B[0-9]*") \
      .copyProperties(image, ["system:time_start"])
def Sn_maskL8sr(image):
    qa = image.select('QA60')
# Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
# Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])
def L7_maskL8sr(image):
      #<<表示二进制左移几位
      # 二进制第三位表示有云阴影，第五位表示有云。
    cloudShadowBitMask = 1 << 4
    cloudsBitMask = 1 << 3
    #选择质量评估波段
    qa = image.select('QA_PIXEL')
    #按位与运算，将检测出云以及云阴影像元值为0
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)\
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    #掩膜云以及云阴影，并选择波段1-9将单位变为0-1之间，并复制影像时间属性。
    return image.updateMask(mask) \
      .select("SR_B[0-9]*") \
      .copyProperties(image, ["system:time_start"])                      
def L5_maskL8sr(image):
      #<<表示二进制左移几位
      # 二进制第三位表示有云阴影，第五位表示有云。
    cloudShadowBitMask = 1 << 4
    cloudsBitMask = 1 << 3
    #选择质量评估波段
    qa = image.select('QA_PIXEL')
    #按位与运算，将检测出云以及云阴影像元值为0
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)\
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    #掩膜云以及云阴影，并选择波段1-9将单位变为0-1之间，并复制影像时间属性。
    return image.updateMask(mask) \
      .select("SR_B[0-9]*") \
      .copyProperties(image, ["system:time_start"]) 

'''定义计算Scikit-learn/8,L7,L5,哨兵卫星的NDVI函数'''
def Cal_L8_NDVI(image):
    ndvi = image.normalizedDifference(["SR_B5","SR_B4"]).rename('RNDVI')
    return ndvi
def Cal_Sn_NDVI(image):
    ndvi = image.normalizedDifference(["B8","B4"]).rename('RNDVI')
    return ndvi.copyProperties(image, ["system:time_start"])
def Cal_L7_NDVI(image):
    ndvi = image.normalizedDifference(["SR_B4","SR_B3"]).rename('RNDVI')
    return ndvi
def Cal_L5_NDVI(image):
    ndvi = image.normalizedDifference(["SR_B4","SR_B3"]).rename('RNDVI')
    return ndvi

'''定义计算L8,L7,L5,哨兵卫星的EVI函数'''
def Cal_L8_EVI(image):
    evi = image.expression(
    '2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
    {
        'red': image.select('SR_B4'),    # 620-670nm, RED
        'nir': image.select('SR_B5'),    # 841-876nm, NIR
        'blue': image.select('SR_B2')    # 459-479nm, BLUE
    }).rename('REVI')
    return evi.copyProperties(image, ["system:time_start"])
def Cal_Sn_EVI(image):
    evi = image.expression(
    '2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
    {
        'red': image.select('B4'),    # 620-670nm, RED
        'nir': image.select('B8'),    # 841-876nm, NIR
        'blue': image.select('B2')    # 459-479nm, BLUE
    }).rename('REVI')
    return evi.copyProperties(image, ["system:time_start"])
def Cal_L7_EVI(image):
    evi = image.expression(
    '2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
        'blue': image.select('SR_B1')    # 459-479nm, BLUE
    }).rename('REVI')
    return evi.copyProperties(image, ["system:time_start"])
def Cal_L5_EVI(image):
    evi = image.expression(
    '2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
        'blue': image.select('SR_B1')    # 459-479nm, BLUE
    }).rename('REVI')
    return evi.copyProperties(image, ["system:time_start"])

'''定义计算L8,L7,L5,哨兵卫星的LSWI函数'''
def Cal_L8_LSWI(image):
    lswi = image.normalizedDifference(["SR_B5","SR_B6"]).rename('RLSWI')
    return lswi.copyProperties(image, ["system:time_start"])
def Cal_Sn_LSWI(image):
    lswi = image.normalizedDifference(["B8","B11"]).rename('RLSWI')
    return lswi.copyProperties(image, ["system:time_start"])
def Cal_L7_LSWI(image):
    lswi = image.normalizedDifference(["SR_B4","SR_B5"]).rename('RLSWI')
    return lswi.copyProperties(image, ["system:time_start"])
def Cal_L5_LSWI(image):
    lswi = image.normalizedDifference(["SR_B4","SR_B5"]).rename('RLSWI')
    return lswi.copyProperties(image, ["system:time_start"])

'''定义计算L8,L7,L5,哨兵卫星的NDWI函数'''
def Cal_L8_NDWI(image):
    ndwi = image.normalizedDifference(['SR_B3','SR_B5']).rename('RNDWI')
    return ndwi.copyProperties(image, ["system:time_start"])
def Cal_Sn_NDWI(image):
    ndwi = image.normalizedDifference(['B3','B8']).rename('RNDWI')
    return ndwi.copyProperties(image, ["system:time_start"])
def Cal_L7_NDWI(image):
    ndwi = image.normalizedDifference(['SR_B2','SR_B4']).rename('RNDWI')
    return ndwi.copyProperties(image, ["system:time_start"])
def Cal_L5_NDWI(image):
    ndwi = image.normalizedDifference(['SR_B2','SR_B4']).rename('RNDWI')
    return ndwi.copyProperties(image, ["system:time_start"])

'''定义计算L8,L7,L5,哨兵卫星的GCVI函数'''
def Cal_L8_GCVI(image):
    gcvi = image.expression(
    'nir / green - 1',
    {
        'nir': image.select('SR_B5'),    
        'green': image.select('SR_B3'),    
    }).rename('RGCVI')
    return gcvi.copyProperties(image, ["system:time_start"])
def Cal_Sn_GCVI(image):
    gcvi = image.expression(
    'nir / green - 1',
    {
        'nir': image.select('B8'),    
        'green': image.select('B3'),    
    }).rename('RGCVI')
    return gcvi.copyProperties(image, ["system:time_start"])
def Cal_L7_GCVI(image):
    gcvi = image.expression(
    'nir / green - 1',
    {
        'nir': image.select('SR_B4'),    
        'green': image.select('SR_B2'),    
    }).rename('RGCVI')
    return gcvi.copyProperties(image, ["system:time_start"])
def Cal_L5_GCVI(image):
    gcvi = image.expression(
    'nir / green - 1',
    {
        'nir': image.select('SR_B4'),    
        'green': image.select('SR_B2'),    
    }).rename('RGCVI')
    return gcvi.copyProperties(image, ["system:time_start"])

'''定义计算L8,L7,L5,哨兵卫星的RVI函数'''
def Cal_L8_RVI(image):
    rvi = image.expression(
    'nir / red',
    {
        'nir': image.select('SR_B5'),    
        'red': image.select('SR_B4'),    
    }).rename('RRVI')
    return rvi.copyProperties(image, ["system:time_start"])
def Cal_Sn_RVI(image):
    rvi = image.expression(
    'nir / red',
    {
        'nir': image.select('B8'),    
        'red': image.select('B4'),    
    }).rename('RRVI')
    return rvi.copyProperties(image, ["system:time_start"])
def Cal_L7_RVI(image):
    rvi = image.expression(
    'nir / red',
    {
        'nir': image.select('SR_B4'),    
        'red': image.select('SR_B3'),    
    }).rename('RRVI')
    return rvi.copyProperties(image, ["system:time_start"])
def Cal_L5_RVI(image):
    rvi = image.expression(
    'nir / red',
    {
        'nir': image.select('SR_B4'),    
        'red': image.select('SR_B3'),    
    }).rename('RRVI')
    return rvi.copyProperties(image, ["system:time_start"])

'''定义计算L8,L7,L5,哨兵卫星的EVI2函数'''
def Cal_L8_EVI2(image):
    evi2 = image.expression(
    '2.5 * (nir - red) / (nir + 2.4 * red  + 1)',
    {
        'red': image.select('SR_B4'),    # 620-670nm, RED
        'nir': image.select('SR_B5'),    # 841-876nm, NIR
    }).rename('REVI2')
    return evi2.copyProperties(image, ["system:time_start"])
def Cal_Sn_EVI2(image):
    evi2 = image.expression(
    '2.5 * (nir - red) / (nir + 2.4 * red  + 1)',
    {
        'red': image.select('B4'),    # 620-670nm, RED
        'nir': image.select('B8'),    # 841-876nm, NIR
    }).rename('REVI2')
    return evi2.copyProperties(image, ["system:time_start"])
def Cal_L7_EVI2(image):
    evi2 = image.expression(
    '2.5 * (nir - red) / (nir + 2.4 * red  + 1)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
    }).rename('REVI2')
    return evi2.copyProperties(image, ["system:time_start"])
def Cal_L5_EVI2(image):
    evi2 = image.expression(
    '2.5 * (nir - red) / (nir + 2.4 * red  + 1)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
    }).rename('REVI2')
    return evi2.copyProperties(image, ["system:time_start"])

'''定义计算L8,L7,L5,哨兵卫星的SAVI函数'''
def Cal_L8_SAVI(image):
    savi = image.expression(
    '1.5 * (nir - red) / (nir +  red  + 0.5)',
    {
        'red': image.select('SR_B4'),    # 620-670nm, RED
        'nir': image.select('SR_B5'),    # 841-876nm, NIR
    }).rename('RSAVI')
    return savi.copyProperties(image, ["system:time_start"])
def Cal_Sn_SAVI(image):
    savi = image.expression(
    '1.5 * (nir - red) / (nir +  red  + 0.5)',
    {
        'red': image.select('B4'),    # 620-670nm, RED
        'nir': image.select('B8'),    # 841-876nm, NIR
    }).rename('RSAVI')
    return savi.copyProperties(image, ["system:time_start"])
def Cal_L7_SAVI(image):
    savi = image.expression(
    '1.5 * (nir - red) / (nir +  red  + 0.5)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
    }).rename('RSAVI')
    return savi.copyProperties(image, ["system:time_start"])
def Cal_L5_SAVI(image):
    savi = image.expression(
    '1.5 * (nir - red) / (nir +  red  + 0.5)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
    }).rename('RSAVI')
    return savi.copyProperties(image, ["system:time_start"])

'''定义计算L8,L7,L5,哨兵卫星的MNDWI函数'''
def Cal_L8_MNDWI(image):
    ndvi = image.normalizedDifference(["SR_B3","SR_B5"]).rename('RMNDWI')
    return ndvi.copyProperties(image, ["system:time_start"])
def Cal_Sn_MNDWI(image):
    ndvi = image.normalizedDifference(["B3","B8"]).rename('RMNDWI')
    return ndvi.copyProperties(image, ["system:time_start"])
def Cal_L7_MNDWI(image):
    ndvi = image.normalizedDifference(["SR_B2","SR_B4"]).rename('RMNDWI')
    return ndvi.copyProperties(image, ["system:time_start"])
def Cal_L5_MNDWI(image):
    ndvi = image.normalizedDifference(["SR_B2","SR_B4"]).rename('RMNDWI')
    return ndvi.copyProperties(image, ["system:time_start"])


def addBands_L8(image):
    NDVI = Cal_L8_NDVI(image)
    EVI = Cal_L8_EVI(image)
    mNDWI = Cal_L8_MNDWI(image)
    return image.addBands(NDVI).addBands(EVI).addBands(mNDWI)
def addBands_L7(image):
    NDVI = Cal_L7_NDVI(image)
    EVI = Cal_L7_EVI(image)
    mNDWI = Cal_L7_MNDWI(image)
    return image.addBands(NDVI).addBands(EVI).addBands(mNDWI)
def addBands_L5(image):
    NDVI = Cal_L5_NDVI(image)
    EVI = Cal_L5_EVI(image)
    mNDWI = Cal_L5_MNDWI(image)
    return image.addBands(NDVI).addBands(EVI).addBands(mNDWI)
def addBands_Sn(image):
    NDVI = Cal_Sn_NDVI(image)
    EVI = Cal_Sn_EVI(image)
    mNDWI = Cal_Sn_MNDWI(image)
    return image.addBands(NDVI).addBands(EVI).addBands(mNDWI)


'''定义区分像元为水像元的函数'''
def Water(image):
    return image.select('RMNDWI').gt(image.select('REVI')).Or(image.select('RMNDWI').gt(image.select('RNDVI'))).And(image.select('REVI').lt(0.1)).rename('Water')


'''定义区分像元为非水像元的函数'''
def NonWater(image):
    return image.select('RMNDWI').lte(image.select('REVI')).And(image.select('RMNDWI').lte(image.select('RNDVI'))).Or(image.select('REVI').gte(0.1)).rename('NonWater')


'''定义随机森林的函数'''
def RF(image,sample,Treenums:int,ClassProperty:str,threshold_value:float,csv_path:str,name:str,classVis,roi,year):
    '''
    Args:
    image: Image which have all classify bands  propertys .
    sample:  Points which  have all classify bands propertys and lucc property  .
    Treenums:  Tree nums
    ClassProperty: LUCC band name
    threshold_value: threshold_value
    InputProperties: Classify bands names(image.bandNames())
    csv_path: csv_path
    name: image name
    classVis； classVis
    roi: roi
    '''
    import csv
    import datetime
        
    sample = sample.randomColumn()    #将采样点打乱顺序
    trainingSample = sample.filter('random <= ' + str(threshold_value))  #获取阈值前的一部分样本点
    validationSample = sample.filter('random > ' + str(threshold_value))   #获取阈值后的一部分样本点
    logging.info(f' {name} Bandnames:  {image.bandNames().getInfo()}')   
    f = open(csv_path,'a',encoding='utf-8') #打开csv数据
    csv_writer = csv.writer(f)  #写入数据            
    trainedClassifier = ee.Classifier.smileRandomForest(Treenums).train(features = trainingSample,\ 
    classProperty = ClassProperty,inputProperties = image.bandNames())    #对训练样本进行随机森林训练 
    trainAccuracy = trainedClassifier.confusionMatrix() #计算混淆矩阵
    start = datetime.datetime.now()
    csv_writer.writerow(['当前时间为： ',start])
    validationSample = validationSample.classify(trainedClassifier)   #对验证样本进行随机森林训练
    validationAccuracy = validationSample.errorMatrix(ClassProperty, 'classification')  #计算混淆矩阵
    csv_writer.writerow(['Validation error matrix_RF_'+ name + '_'+str(year) +'_'+str(Treenums) +': ', str(validationAccuracy.getInfo())])
    csv_writer.writerow(['Validation overall accuracy_RF_'+ name + '_'+str(year) +'_'+str(Treenums) +': ', str(validationAccuracy.accuracy().getInfo())])
    csv_writer.writerow(['Validation kappa accuracy_RF_' + name + '_'+str(year) +'_'+str(Treenums) +': ', str(validationAccuracy.kappa().getInfo())])
    logging.info('Validation accuracy_RF_' + name + '_'+str(year) +'_'+str(Treenums) +': ' + str(validationAccuracy.accuracy().getInfo()))
    logging.info('Validation kappa accuracy_RF_' + name + '_'+str(year) +'_'+str(Treenums) +': ' + str(validationAccuracy.kappa().getInfo()))
    imgClassified = image.classify(trainedClassifier)
    crop = imgClassified.eq(3).selfMask()   #提取属于水稻的像元
    pixel_area = crop.multiply(ee.Image.pixelArea()).divide(1e6)   #计算每个像元的面积
    #区域统计全部水稻的面积
    img_area = pixel_area.reduceRegion(**{
        'reducer':ee.Reducer.sum(),
        'geometry':roi,
        'scale':30,
        'maxPixels':1e13
    })
    print('(img_area.getInfo()',img_area.getInfo()['classification'])
    csv_writer.writerow(['Validation kappa accuracy_RF_' + name + '_'+ str(year) +'_'+str(Treenums) +'水稻面积为(Km): ', img_area.getInfo()['classification']])
    f.close()
    Map.addLayer(imgClassified, classVis, 'Classified_RF' + '_' + name + '_'+ str(year), False)
    return imgClassified

'''定义支持向量机的函数'''
def SVM(image,sample,ClassProperty:str,threshold_value:float,csv_path:str,name:str,classVis,roi):
    '''
    Args:
    image: Image which have all classify bands  propertys .
    sample:  Points which  have all classify bands propertys and lucc property  .
    ClassProperty: LUCC band name
    threshold_value: threshold_value
    InputProperties: Classify bands names(image.bandNames())
    csv_path: csv_path
    name: image name
    classVis； classVis
    '''
    import datetime
    import csv   
    sample = sample.randomColumn()   
    trainingSample = sample.filter('random <= ' + str(threshold_value))
    validationSample = sample.filter('random > ' + str(threshold_value))  
    logging.info(f' {name} Bandnames:  {image.bandNames().getInfo()}')   
    f = open(csv_path,'a',encoding='utf-8')
    csv_writer = csv.writer(f)
    trainedClassifier = ee.Classifier.libsvm().train(features = trainingSample,classProperty = ClassProperty,inputProperties = image.bandNames())
    trainAccuracy = trainedClassifier.confusionMatrix()
    start = datetime.datetime.now()
    logging.info('Training overall accuracy_SVM '+ name + ': ' +  str(trainAccuracy.accuracy().getInfo()))
    logging.info('Training kappa accuracy_SVM ' + name + ': ' + str(trainAccuracy.kappa().getInfo()))
    validationSample = validationSample.classify(trainedClassifier)
    validationAccuracy = validationSample.errorMatrix(ClassProperty, 'classification')
    logging.info('Validation accuracy_SVM ' + name + ': ' + str(validationAccuracy.accuracy().getInfo()))
    logging.info('Validation kappa accuracy_SVM ' + name + ': ' + str(validationAccuracy.kappa().getInfo()))
    imgClassified = image.classify(trainedClassifier)
    CropArea = pixelArea.updateMask(Crop)
    area = CropArea.reduceRegion(reducer =  ee.Reducer.sum(),geometry = roi,maxPixels = 1e8,scale =  30)
    print('area.getInfo',area.getInfo())
    f.close()
    Map.addLayer(imgClassified, classVis, 'Classified_SVM' + name, False)
    return imgClassified

I_list = ['RNDVI','REVI','RLSWI','RNDWI','RGCVI','RRVI','RSAVI']
Cal_L8_list = [Cal_L8_NDVI,Cal_L8_EVI,Cal_L8_LSWI,Cal_L8_NDWI,Cal_L8_GCVI,Cal_L8_RVI,Cal_L8_SAVI]
Cal_L7_list = [Cal_L7_NDVI,Cal_L7_EVI,Cal_L7_LSWI,Cal_L7_NDWI,Cal_L7_GCVI,Cal_L7_RVI,Cal_L7_SAVI]
Cal_L5_list = [Cal_L5_NDVI,Cal_L5_EVI,Cal_L5_LSWI,Cal_L5_NDWI,Cal_L5_GCVI,Cal_L5_RVI,Cal_L5_SAVI]
Cal_Sn_list = [Cal_Sn_NDVI,Cal_Sn_EVI,Cal_Sn_LSWI,Cal_Sn_NDWI,Cal_Sn_GCVI,Cal_Sn_RVI,Cal_Sn_SAVI]
sg.popup_notify(title = 'Task done!',display_duration_in_ms = 100,fade_in_duration = 100)

In [ ]:
'''Main'''
sy = 2016
ey = 2016

treenums = 30
'''Processing'''
'''循环每年的数据'''
for yyear in range(2018,2022):
    '''循环每个时期的数据'''
    for Da,name1 in tqdm(zip(D_list,Names),desc = '时期',colour  = 'white'):  #循环每个时期
        all_cols = ee.Image()  #将所有指数最后放在一个影像中
        '''循环每种指数'''
        for Cal_L8,Cal_L7,Cal_L5,Cal_Sn,I in zip(Cal_L8_list,Cal_L7_list,Cal_L5_list,Cal_Sn_list,I_list):   #循环每个指数
            cols_list = ee.List([])  #将每年同一时期的影像放在一起
                '''过滤出符合要求的数据'''
                L_8 = Landsat8_col.filterDate(str(year) + '-' + Da[0],str(yyear) + '-' + Da[1])\
                .filterBounds(roi).map(L8_applyScaleFactors).select("SR_B.*|QA_P.*").map(clipp).map(L8_maskL8sr).map

                L_7 = Landsat7_col.filterDate(str(year) + '-' + Da[0],str(yyear) + '-' + Da[1])\
                .filterBounds(roi).map(L7_applyScaleFactors).select("SR_B.*|QA_P.*").map(clipp).map(L7_maskL8sr)

                L_5 = Landsat5_col.filterDate(str(year) + '-' + Da[0],str(yyear) + '-' + Da[1])\
                .filterBounds(roi).map(L5_applyScaleFactors).select("SR_B.*|QA_P.*").map(clipp).map(L5_maskL8sr)

                Sen = Sentinel_col.filterDate(str(year) + '-' + Da[0],str(yyear) + '-' + Da[1])\
                .filterBounds(roi).map(Sn_applyScaleFactors).select("B.*|QA60").map(clipp).map(Sn_maskL8sr)
                
                '''对每种数据计算对应的指数'''
                L_8_I = L_8.map(Cal_L8).qualityMosaic(I)   
                L_7_I = L_7.map(Cal_L7).qualityMosaic(I)   
                L_5_I = L_5.map(Cal_L5).qualityMosaic(I)   
                Sn_I = Sen.map(Cal_Sn).qualityMosaic(I)   
                cols_list = cols_list.add(L_8_I).add(L_7_I).add(Sn_I).add(L_5_I)  
            cols = ee.ImageCollection.fromImages(cols_list)   #将列表转为影像集
            logging.info(f'{Da}-{I}-SIZE: {cols.size().getInfo()}')  #查看张数，每年每一时期一张
            col = cols.qualityMosaic(I)  #多年同一时期融合成有值的一张影像
            all_cols = all_cols.addBands(col)  #添加波段
        all_cols = all_cols.select("R.*").addBands(SRTM30).addBands(slope)   #添加dem和slope辅助数据
        logging.info('————————————————Processing————————————————————')
        sample = all_cols.addBands(LUCC).stratifiedSample(numPoints=100,classBand=label,region= roi,scale= 5,geometries= True)  #创建采样点
        cd_RF = RF(image = all_cols,sample = sample,Treenums = treenums,ClassProperty = label,threshold_value = 0.8,\
                csv_path = '竞赛.csv',name = name1,classVis = classVis,roi = roi,year  = yyear)#运用随机森林函数
        logging.info(f'{name1} is ok !!!!!!')
    logging.info('⚡️⚡️⚡️⚡️⚡️')
    sg.popup_notify(title = 'Task done!',display_duration_in_ms = 1000,fade_in_duration = 1000)

In [ ]:
'''Processing  Water'''
for Da,name1 in tqdm(zip(D_list,Names),desc = '时期',colour  = 'white'):  #循环每个时期
    list_water = ee.List([])   #存放水像元的列表
    list_nonwater = ee.List([])  #存放非水像元的列表
    for yyear in tqdm(range(2013,2022)):
        L_8 = Landsat8_col.filterDate(str(yyear) + '-' + Da[0],str(yyear) + '-' + Da[1])\
        .filterBounds(roi).map(L8_applyScaleFactors).select("SR_B.*|QA_P.*").map(clipp).map(L8_maskL8sr).map(addBands_L8)
        L_8_Water = L_8.map(Water).mosaic()
        L_8_NoWater = L_8.map(NonWater).mosaic()
        
        L_7 = Landsat7_col.filterDate(str(yyear) + '-' + Da[0],str(yyear) + '-' + Da[1])\
        .filterBounds(roi).map(L7_applyScaleFactors).select("SR_B.*|QA_P.*").map(clipp).map(L7_maskL8sr).map(addBands_L7)
        L_7_Water = L_7.map(Water).mosaic()
        L_7_NoWater = L_7.map(NonWater).mosaic()
        
        L_5 = Landsat5_col.filterDate(str(yyear) + '-' + Da[0],str(yyear) + '-' + Da[1])\
        .filterBounds(roi).map(L5_applyScaleFactors).select("SR_B.*|QA_P.*").map(clipp).map(L5_maskL8sr).map(addBands_L5)
        L_5_Water = L_5.map(Water).mosaic()
        L_5_NoWater = L_5.map(NonWater).mosaic()
        
        Sen = Sentinel_col.filterDate(str(yyear) + '-' + Da[0],str(yyear) + '-' + Da[1])\
        .filterBounds(roi).map(Sn_applyScaleFactors).select("B.*|QA60").map(clipp).map(Sn_maskL8sr).map(addBands_Sn)
        L_Sn_Water = Sen.map(Water).mosaic()
        L_Sn_NoWater = Sen.map(NonWater).mosaic()
        
        all_water =  ee.List([]).add(L_8_Water).add(L_7_Water).add(L_5_Water).add(L_Sn_Water)  
        all_nonwater = ee.List([]).add(L_8_NoWater).add(L_7_NoWater).add(L_5_NoWater).add(L_Sn_NoWater)  
        water = ee.ImageCollection.fromImages(all_water).mosaic()   
        nonwater = ee.ImageCollection.fromImages(all_nonwater).mosaic()
        
        list_water = list_water.add(water)
        list_nonwater = list_nonwater.add(nonwater)
    water = ee.ImageCollection.fromImages(list_water).sum()  #计算属于水像元的天数
    print('water',water.getInfo())
    nonwater = ee.ImageCollection.fromImages(list_nonwater).sum()  #计算属于非水像元的天数
    print('nonwater',nonwater.getInfo())
    flooding_freq = water.divide(water.add(nonwater)).rename("flooding_freq")  #计算淹没频次
    print('flooding_freq',flooding_freq.getInfo())
    Map.addLayer(flooding_freq, {}, name1 + '_flooding_freq', True)   
    #将图像导出
    geemap.ee_export_image_to_drive(flooding_freq,description = name1 + '_flooding_freq' ,crs =  "EPSG:4326",scale =  30,region =  roi,folder='竞赛')
    logging.info(f'{name1} is ok !!!!!!')
logging.info('⚡️⚡️⚡️⚡️⚡️')
sg.popup_notify(title = 'Task done!',display_duration_in_ms = 100,fade_in_duration = 100)